#### Instalar twilio

In [30]:
#!pip3 install twilio

In [31]:
#!pip3 install twilio requests pandas beautifulsoup4 tqdm

In [32]:
# puedes usar el .env para las variables de configuracion de twilio o usar un archivo twilio_config y a ese archivo aplicar las variables del .env
#!pip3 install python-dotenv

In [33]:
import os
from twilio.rest import Client
from twilio_config import *
#from dotenv import load_dotenv
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

# Cargar variables de entorno desde .env
#load_dotenv()

# Obtener las variables de entorno
#TWILIO_ACCOUNT_SID = os.getenv('TWILIO_ACCOUNT_SID')
#TWILIO_AUTH_TOKEN = os.getenv('TWILIO_AUTH_TOKEN')
#PHONE_NUMBER = os.getenv('PHONE_NUMBER')
#API_KEY_WAPI = os.getenv('API_KEY_WAPI')


#### URL API

In [34]:
query = 'limache'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=644d3d8e0de44e728b5234001251003&q=limache&days=1&aqi=no&alerts=no'

In [35]:
response = requests.get(url_clima).json()
response

{'location': {'name': 'Limache',
  'region': 'Valparaiso',
  'country': 'Chile',
  'lat': -32.9833,
  'lon': -71.2833,
  'tz_id': 'America/Santiago',
  'localtime_epoch': 1741803923,
  'localtime': '2025-03-12 15:25'},
 'current': {'last_updated_epoch': 1741803300,
  'last_updated': '2025-03-12 15:15',
  'temp_c': 20.4,
  'temp_f': 68.7,
  'is_day': 1,
  'condition': {'text': 'Sunny',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
   'code': 1000},
  'wind_mph': 6.9,
  'wind_kph': 11.2,
  'wind_degree': 295,
  'wind_dir': 'WNW',
  'pressure_mb': 1016.0,
  'pressure_in': 30.0,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 64,
  'cloud': 0,
  'feelslike_c': 20.4,
  'feelslike_f': 68.7,
  'windchill_c': 20.8,
  'windchill_f': 69.4,
  'heatindex_c': 20.8,
  'heatindex_f': 69.4,
  'dewpoint_c': 12.4,
  'dewpoint_f': 54.2,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 8.3,
  'gust_mph': 8.0,
  'gust_kph': 12.9},
 'forecast': {'forecastday': [{'date': '2025-03-12',
    '

In [36]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [37]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [38]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [39]:
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1741748400,
 'time': '2025-03-12 00:00',
 'temp_c': 15.3,
 'temp_f': 59.5,
 'is_day': 0,
 'condition': {'text': 'Clear ',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
  'code': 1000},
 'wind_mph': 3.1,
 'wind_kph': 5.0,
 'wind_degree': 190,
 'wind_dir': 'S',
 'pressure_mb': 1015.0,
 'pressure_in': 29.97,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'snow_cm': 0.0,
 'humidity': 69,
 'cloud': 0,
 'feelslike_c': 15.3,
 'feelslike_f': 59.5,
 'windchill_c': 15.3,
 'windchill_f': 59.5,
 'heatindex_c': 15.3,
 'heatindex_f': 59.5,
 'dewpoint_c': 9.6,
 'dewpoint_f': 49.4,
 'will_it_rain': 0,
 'chance_of_rain': 0,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 6.0,
 'gust_kph': 9.6,
 'uv': 0}

In [40]:
response['forecast']['forecastday'][0]['hour'][0]['time']

'2025-03-12 00:00'

In [41]:
# fecha 
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

'2025-03-12'

In [42]:
# hora del dia 
int (response['forecast']['forecastday'][0]['hour'][0]['time'].split()[1].split(':')[0])

0

In [43]:
# condicion del clima
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']

'Clear '

In [44]:
# tenperatura en celsius
response['forecast']['forecastday'][0]['hour'][0]['temp_c']

15.3

In [45]:
# will_it_rain nos dice si llovera o no (1;0)
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [46]:
# probabilidad de lluvia %
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

0

### Creando mi DATAFRAME

In [47]:
# funcion para obtener todos los datos necesarios
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [48]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))

100%|██████████| 24/24 [00:00<00:00, 117050.34it/s]


In [49]:
datos

[('2025-03-12', 0, 'Clear ', 15.3, 0, 0),
 ('2025-03-12', 1, 'Clear ', 15.2, 0, 0),
 ('2025-03-12', 2, 'Clear ', 15.0, 0, 0),
 ('2025-03-12', 3, 'Clear ', 14.9, 0, 0),
 ('2025-03-12', 4, 'Clear ', 14.9, 0, 0),
 ('2025-03-12', 5, 'Clear ', 14.5, 0, 0),
 ('2025-03-12', 6, 'Clear ', 14.6, 0, 0),
 ('2025-03-12', 7, 'Clear ', 14.8, 0, 0),
 ('2025-03-12', 8, 'Sunny', 16.9, 0, 0),
 ('2025-03-12', 9, 'Sunny', 18.5, 0, 0),
 ('2025-03-12', 10, 'Sunny', 19.7, 0, 0),
 ('2025-03-12', 11, 'Sunny', 20.5, 0, 0),
 ('2025-03-12', 12, 'Sunny', 20.9, 0, 0),
 ('2025-03-12', 13, 'Sunny', 21.3, 0, 0),
 ('2025-03-12', 14, 'Sunny', 21.1, 0, 0),
 ('2025-03-12', 15, 'Sunny', 20.4, 0, 0),
 ('2025-03-12', 16, 'Sunny', 20.5, 0, 0),
 ('2025-03-12', 17, 'Sunny', 20.1, 0, 0),
 ('2025-03-12', 18, 'Sunny', 19.0, 0, 0),
 ('2025-03-12', 19, 'Sunny', 17.4, 0, 0),
 ('2025-03-12', 20, 'Sunny', 16.6, 0, 0),
 ('2025-03-12', 21, 'Clear ', 16.6, 0, 0),
 ('2025-03-12', 22, 'Clear ', 16.4, 0, 0),
 ('2025-03-12', 23, 'Clear ', 16.2

In [50]:
# DATAFRAME

col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2025-03-12,0,Clear,15.3,0,0
1,2025-03-12,1,Clear,15.2,0,0
2,2025-03-12,2,Clear,15.0,0,0
3,2025-03-12,3,Clear,14.9,0,0
4,2025-03-12,4,Clear,14.9,0,0
5,2025-03-12,5,Clear,14.5,0,0
6,2025-03-12,6,Clear,14.6,0,0
7,2025-03-12,7,Clear,14.8,0,0
8,2025-03-12,8,Sunny,16.9,0,0
9,2025-03-12,9,Sunny,18.5,0,0


In [51]:
# FILTRAMOS SOLO SI LLUEVE
#df[df['Lluvia']==1]

In [52]:
# FILTRAMOS LAS HORAS QUE NECESITAMOS y si queremos podemos crear otro filtro para una consicion climatica en especial en este caso solo filtramos un rango de horas

df_rain =  df[(df['Hora']>10) & (df['Hora']< 19)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)
df_rain

,Condicion
Hora,
11,Sunny
12,Sunny
13,Sunny
14,Sunny
15,Sunny
16,Sunny
17,Sunny
18,Sunny


In [53]:
# ejemplo de mensaje que llegara al telefono (no llegara asi de desordenado, llegara formteado)
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2025-03-12 en limache es : \n\n\n      Condicion\nHora          \n11       Sunny\n12       Sunny\n13       Sunny\n14       Sunny\n15       Sunny\n16       Sunny\n17       Sunny\n18       Sunny'

In [54]:
# numero de telefono que compramos para hacer las pruebas
PHONE_NUMBER

'+1 262 384 6544'

#### ENVINDO MENSAJE SMS CON TWILIO

In [55]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN
client = Client(account_sid, auth_token)

message = client.messages.create(
    from_='+12623846544',
    body='\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
    to='+56965747898'
)

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SMf22419c3504402199ee0ea9f44cc28bf
